In [ ]:
import numpy as np
import pandas as pd
import networkx as network
import matplotlib.pyplot as plot
from hmmlearn.hmm import CategoricalHMM

In [ ]:
visibleStates = ['early', 'mid', 'late']
hiddenStates = ['Genuine User', 'Intruder']

hiddenInitial = [0.9, 0.1]
visibleInitial = [0.33, 0.33, 0.33]

hiddenTransition = pd.DataFrame([
    [0.7, 0.3],
    [0.4, 0.6]
], columns=hiddenStates, index=hiddenStates)

emissionMatrix = pd.DataFrame([
    [0.8, 0.1, 0.1],
    [0.1, 0.3, 0.6]
], columns=visibleStates, index=hiddenStates)

In [ ]:
hiddenTransition

In [ ]:
emissionMatrix

In [ ]:
transitionArray = hiddenTransition.values
emissionArray = emissionMatrix.values

In [ ]:
def edgeDictionary(dataframe):
    return {(row, column): dataframe.loc[row, column] 
            for row in dataframe.index 
            for column in dataframe.columns}

In [ ]:
hiddenEdges = edgeDictionary(hiddenTransition)
emissionEdges = edgeDictionary(emissionMatrix)

In [ ]:
graph = network.DiGraph()
graph.add_nodes_from(visibleStates + hiddenStates)

for key, value in {**hiddenEdges, **emissionEdges}.items():
    graph.add_edge(key[0], key[1], weight=value, label=f"{value:.2f}")

position = network.circular_layout(graph)

In [ ]:
nodeColors = ['skyblue' if node in hiddenStates else 'lightgreen' for node in graph]
network.draw(graph, position, with_labels=True, arrows=True, node_color=nodeColors, node_size=1000, font_size=7, font_weight='bold')
network.draw_networkx_edge_labels(graph, position, edge_labels=network.get_edge_attributes(graph, 'label'), font_size=8)
plot.title("HMM State and Emission Graph")
plot.show()

In [ ]:
observations = ['early', 'early', 'late', 'mid', 'early', 'late']
observationMap = {'early': 0, 'mid': 1, 'late': 2}
mappedSequence = np.array([observationMap[value.lower()] for value in observations]).reshape(-1, 1)

In [ ]:
model = CategoricalHMM(n_components=len(hiddenStates), init_params="")
model.startprob_ = np.array(hiddenInitial)
model.transmat_ = transitionArray
model.emissionprob_ = emissionArray
model.n_features = len(visibleStates)

In [ ]:
logValue, bestPath = model.decode(mappedSequence, algorithm="viterbi")

stateMap = {index: state for index, state in enumerate(hiddenStates)}
decodedPath = [stateMap[state] for state in bestPath]

In [ ]:
resultFrame = pd.DataFrame({'Observation': observations, 'Predicted State': decodedPath})
print("\nDecoded Path with Observations:")
print(resultFrame)

print("\nLog Probability of Best Path:", logValue)